# Загрузка библиотек

In [85]:
import csv
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

# Обработка данных

В процессе работы было принято решение сократить набор данных с 5240 классов до 10, так как не получилось уложиться во временные рамки

In [103]:
# Загрузка большого и маленького датасетов
big_dataset = pd.read_csv('minerals_full.csv')
small_dataset = pd.read_csv('minerals_10.csv')

# Оставляем только строки большого датасета, у которых id есть в маленьком датасете
filtered_dataset = big_dataset[big_dataset['id'].isin(small_dataset['id'])]

# Сохранение отфильтрованного датасета
filtered_dataset.to_csv('minerals_10_full.csv', index=False)

## Проверка полученного датасета

In [106]:
df = pd.read_csv('minerals_10_full.csv', parse_dates=True)

In [113]:
df.head(164)

,id,path,height,width,ru_name,ru_synonim_name,ru_variety,ru_satellites,ru_description,en_name,en_variety,en_satellites,en_description,text_boxes,mineral_boxes,mineral_boxes_parsed,box_coordinates
0,1_1226,data/1_syst/00000/1_1226/IMG_6972.JPG,3024,3780,Гематит,гематит,гематит,NaN,NaN,hematite,NaN,NaN,NaN,"[[[0.82324, 0.83639], [0.86914, 0.84371]], [[0...","[{""label"": ""a mineral"", ""confidence"": 0.103, ""...","[{'label': 'a mineral', 'confidence': 0.103, '...","[[0.03223, 0.01709, 0.98145, 0.85348], [0.0214..."
1,1_126,data/1_syst/00000/1_126/126.jpg,3024,4032,Халцедон,халцедон,халцедон,NaN,NaN,chalcedony,NaN,NaN,NaN,"[[[0.65625, 0.46224], [0.69043, 0.51042]], [[0...","[{""label"": ""a rock"", ""confidence"": 0.398, ""box...","[{'label': 'a rock', 'confidence': 0.398, 'box...","[[0.30762, 0.32292, 0.61426, 0.60807], [0.6035..."
2,1_1299,data/1_syst/00000/1_1299/1_1299.jpg,3456,4608,Касситерит,касситерит,NaN,NaN,NaN,cassiterite,NaN,NaN,NaN,"[[[0.79297, 0.65104], [0.83594, 0.67057]]]","[{""label"": ""a stone"", ""confidence"": 0.401, ""bo...","[{'label': 'a stone', 'confidence': 0.401, 'bo...","[[0.17285, 0.125, 0.7168, 0.80729]]"
3,1_134,data/1_syst/00000/1_134/1_134.jpg,2424,3474,Кварц,кварц,кварц,NaN,NaN,quartz,NaN,NaN,NaN,"[[[0.70312, 0.45938], [0.74609, 0.47479]]]","[{""label"": ""a stone"", ""confidence"": 0.121, ""bo...","[{'label': 'a stone', 'confidence': 0.121, 'bo...","[[0.24512, 0.33894, 0.64258, 0.5042], [0.6748,..."
4,1_1495,data/1_syst/00000/1_1495/1_1495.jpg,3311,4415,Кварц,кварц,кварц,NaN,NaN,quartz,NaN,NaN,NaN,"[[[0.68555, 0.73794], [0.7168, 0.75619]]]","[{""label"": ""a mineral"", ""confidence"": 0.117, ""...","[{'label': 'a mineral', 'confidence': 0.117, '...","[[0.0918, 0.14472, 0.82617, 0.73403], [0.09473..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,1_2872,data/1_syst/00000/1_2872/IMG_6274.jpg,3024,2892,Кварц,кварц,кварц,NaN,NaN,quartz,NaN,NaN,NaN,"[[[0.44229, 0.78125], [0.50664, 0.80469]], [[0...","[{""label"": ""a stone"", ""confidence"": 0.199, ""bo...","[{'label': 'a stone', 'confidence': 0.199, 'bo...","[[0.15015, 0.04785, 0.86313, 0.80957]]"
160,1_2873,data/1_syst/00000/1_2873/1_2873.jpg,3111,4931,Кварц,кварц,кварц,NaN,Одтдельный призматический кристалл прозрачного...,quartz,NaN,NaN,single prismatic clear quartz crystal (white a...,[],"[{""label"": ""a mineral"", ""confidence"": 0.117, ""...","[{'label': 'a mineral', 'confidence': 0.117, '...","[[0.17578, 0.21053, 0.94629, 0.66873], [0.1699..."
161,1_2874,data/1_syst/00000/1_2874/1_2874.jpg,2925,4160,Кварц,кварц,кварц,NaN,NaN,quartz,NaN,NaN,NaN,"[[[0.75586, 0.53056], [0.80273, 0.54861]], [[0...","[{""label"": ""a stone"", ""confidence"": 0.425, ""bo...","[{'label': 'a stone', 'confidence': 0.425, 'bo...","[[0.2168, 0.27361, 0.66113, 0.7875]]"
162,1_2875,data/1_syst/00000/1_2875/1_2875.jpg,2777,4325,Кварц,кварц,кварц,NaN,NaN,quartz,NaN,NaN,NaN,"[[[0.7998, 0.47945], [0.84375, 0.49619]], [[0....","[{""label"": ""a stone"", ""confidence"": 0.164, ""bo...","[{'label': 'a stone', 'confidence': 0.164, 'bo...","[[0.11816, 0.29224, 0.69727, 0.81735]]"


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8549 entries, 0 to 8548
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    8549 non-null   object
 1   path                  8549 non-null   object
 2   height                8549 non-null   int64 
 3   width                 8549 non-null   int64 
 4   ru_name               8549 non-null   object
 5   ru_synonim_name       8549 non-null   object
 6   ru_variety            6387 non-null   object
 7   ru_satellites         1281 non-null   object
 8   ru_description        2764 non-null   object
 9   en_name               8549 non-null   object
 10  en_variety            118 non-null    object
 11  en_satellites         778 non-null    object
 12  en_description        2764 non-null   object
 13  text_boxes            8549 non-null   object
 14  mineral_boxes         8549 non-null   object
 15  mineral_boxes_parsed  8549 non-null   

### Проверка корректности пути 

In [16]:
# Функция для чтения CSV файла и фильтрации строк с запятыми в пути к изображениям 
def filter_csv(csv_file_path, output_file_path): 
    filtered_rows = [] 
    encodings = ['utf-8', 'cp1251']  # Список возможных кодировок для попытки чтения файла 
 
    for encoding in encodings: 
        try: 
            with open(csv_file_path, 'r', newline='', encoding=encoding) as csvfile: 
                csvreader = csv.DictReader(csvfile) 
                for row in csvreader: 
                    image_path = row['path'] 
                    if ',' not in image_path: 
                        filtered_rows.append(row) 
            break  # Прерываем цикл, если файл успешно прочитан с текущей кодировкой 
        except UnicodeDecodeError: 
            continue  # Пробуем следующую кодировку, если текущая вызывает ошибку декодирования 
 
    # Запись оставшихся строк в новый файл CSV 
    fieldnames = filtered_rows[0].keys() if filtered_rows else [] 
    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:  # Записываем с кодировкой utf-8 
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames) 
        writer.writeheader() 
        writer.writerows(filtered_rows) 
 
# Пример использования функции для фильтрации CSV файла 
csv_file_path = 'minerals_10_clean.csv' 
output_file_path = 'minerals_10_clean_filtered.csv' 
filter_csv(csv_file_path, output_file_path)

### Изменеие размера изображения

In [5]:
desired_size = (1024, 1024)

# Функция для проверки существования файла
def file_exists(file_path):
    return os.path.exists(file_path)

# Функция для изменения размера изображения с добавлением фона
def resize_image(image_path, output_directory):
    image = Image.open(image_path)
    
    # Создаем новое изображение с фоном, чтобы сохранить соотношение сторон
    new_image = Image.new("RGB", desired_size, (255, 255, 255))  # Белый фон
    image.thumbnail(desired_size)
    
    # Вычисляем положение изображения на фоне
    position = ((desired_size[0] - image.width) // 2, (desired_size[1] - image.height) // 2)
    
    # Накладываем изображение на фон
    new_image.paste(image, position)
    
    # Путь для сохранения измененного изображения
    output_path = os.path.join(output_directory, os.path.basename(image_path))
    
    # Сохраняем результат
    new_image.save(output_path)

# Функция для фильтрации CSV файла по существующим изображениям
def filter_csv_by_existing_images(csv_file_path, output_file_path):
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.DictReader(csvfile)
        rows_to_keep = []
        for row in csvreader:
            image_path = row['path']
            # Используем os.path.normpath для нормализации пути с русскими символами
            image_path = os.path.normpath(image_path)
            if file_exists(image_path):
                rows_to_keep.append(row)

    # Запись оставшихся строк в новый файл CSV
    fieldnames = rows_to_keep[0].keys() if rows_to_keep else []
    with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows_to_keep)

    # Изменяем размер изображений для оставшихся строк
    for row in rows_to_keep:
        image_path = os.path.normpath(row['path'])
        resize_image(image_path, os.path.dirname(image_path))

# Пример использования функции для фильтрации CSV файла и изменения размера изображений
csv_file_path = 'minerals_10_clean.csv'
output_file_path = 'minerals_10_clean_filtered_existing_images.csv'  # Мы перезаписываем исходный файл CSV
filter_csv_by_existing_images(csv_file_path, output_file_path)